# Inserir dados no ElasticSearch - wsl2 (Windows 10)

In [ ]:
# configurar as variáveis de ambiente
import os
import elasticsearch
from elasticsearch import Elasticsearch, helpers
 
# tornar o pyspark "importável"
import findspark
findspark.init('../../anaconda3/spark-2.4.4-bin-hadoop2.7')

In [ ]:
# iniciar uma sessão local
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import functions as f
from pyspark.sql.window import Window as w
import pyodbc
import pandas as pd
import json

appName = "PySpark Test"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster("local") 
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [ ]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [ ]:
es.indices.create(index='andre_elk',ignore=400)

In [ ]:
res=es.indices.get_alias("*")
for Name in res:
    print(Name)

In [ ]:
df_spark = spark.read.option("delimiter", ";").csv("/home/aluisf/estudos/dados/mvno_arquivos/*.txt", header=True)

In [ ]:
df_pd = df_spark.select("CPF_CNPJ","NOME_DO_ASSINANTE","NUMERO_DO_TERMINAL","SITUACAO","DATA_INICIO_DO_VINCULO","DATA_FIN_DO_CANCELAMENTO").toPandas()
df_pd.columns

In [ ]:
df_pd.isna().sum()

In [ ]:
df_pd = df_pd.dropna(subset=["CPF_CNPJ"])

In [ ]:
df_pd.shape

In [ ]:
df_pd2 = df_pd.to_dict('records')

In [ ]:
df_pd2[0]

In [ ]:
def generator(df_pd2):
    for c, line in enumerate(df_pd2):
        yield {
            "_index":"andre_elk",
            "_id":line.get("CPF_CNPJ",None),
            "_source":{
                "nome_do_assinante":line.get("NOME_DO_ASSINANTE",""),
                "numero_do_terminal":line.get("NUMERO_DO_TERMINAL",""),
                "situacao":line.get("SITUACAO",""),
                "data_inicio":line.get("DATA_INICIO_DO_VINCULO",""),
                "data_fim":line.get("DATA_FIN_DO_CANCELAMENTO","")
            }
        }
        raise StopIteration

In [ ]:
elk_test = generator(df_pd2)

In [ ]:
next(elk_test)

In [ ]:
try:
    res = helpers.bulk(es, generator(df_pd2))
    print("Carregando")
except Exception as e:
    print(e)